
# Week07 — DPO(Direct Preference Optimization) 실습 (Local 환경용)

**목표**
- 6주차 SFT(LoRA) 모델을 **초기 정책**으로 활용(있으면)해서, 소형 preference 데이터로 **DPO 학습**을 수행
- **Base/SFT vs DPO** 응답을 동일 프롬프트로 비교·기록
- (선택) 로컬 **Ollama `llama3.1:8b-instruct`**와 간단 비교

> ⚙️ 본 노트북은 **로컬 실행**을 전제로 합니다. GPU가 있으면 4bit/8bit 로딩을 활용하고, CPU만 있어도 작은 모델로 데모가 가능합니다.


## 0) 환경 준비

In [ ]:

# !pip install -q -U transformers datasets peft accelerate bitsandbytes trl sentencepiece #   langfuse python-dotenv pandas requests torch --extra-index-url https://download.pytorch.org/whl/cu121
# ↑ 필요시 주석 해제. CUDA 버전에 맞춰 torch 인덱스는 조정하세요.

import sys, platform, torch, os, json, time, random
from pathlib import Path
print('Python:', sys.version)
print('Platform:', platform.platform())
print('CUDA available:', torch.cuda.is_available())
print('Torch:', torch.__version__)


## 1) .env 로드 (OpenAI/Langfuse/Pinecone 키 등)

In [ ]:

from dotenv import load_dotenv, find_dotenv
from pathlib import Path

env_path = find_dotenv(usecwd=True)
if not env_path:
    cur = Path.cwd()
    for p in [cur] + list(cur.parents):
        cand = p / '.env'
        if cand.exists():
            env_path = str(cand)
            break

if env_path:
    load_dotenv(env_path)
    print(f"Loaded .env from: {env_path}")
else:
    print("⚠️ .env를 찾지 못했습니다. 프로젝트 루트에 .env를 생성/배치하세요.")

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY', '')
LANGFUSE_PUBLIC_KEY = os.getenv('LANGFUSE_PUBLIC_KEY', '')
LANGFUSE_SECRET_KEY = os.getenv('LANGFUSE_SECRET_KEY', '')
LANGFUSE_HOST = os.getenv('LANGFUSE_HOST', 'https://cloud.langfuse.com')
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY', '')

print('OpenAI key set:', bool(OPENAI_API_KEY))
print('Langfuse host:', LANGFUSE_HOST)
print('Pinecone key set:', bool(PINECONE_API_KEY))


## 2) 실험 설정

In [ ]:

from dataclasses import dataclass
from typing import Optional

@dataclass
class CFG:
    # 경로
    data_dir: Path = Path('week07/data')
    out_dir: Path = Path('week07/runs/dpo')
    # 6주차 SFT(LoRA) 어댑터 경로(있으면 사용)
    sft_adapter_dir: Path = Path('week06/runs/lora_sft/adapter')
    base_model: str = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"  # 데모용 소형 모델
    use_qlora: bool = True  # GPU면 4bit 우선
    use_ref_model: bool = False  # 기본은 ref-free DPO
    ref_is_base: bool = True     # True면 ref=base, False면 ref=SFT(있으면)
    # 토큰/길이
    max_length: int = 256
    # DPO 하이퍼파라미터
    beta: float = 0.1
    # 학습 하이퍼파라미터
    lr: float = 5e-6
    num_epochs: int = 1
    per_device_train_batch_size: int = 1
    grad_accum_steps: int = 8
    logging_steps: int = 10
    save_steps: int = 50
    warmup_ratio: float = 0.05
    # 평가 프롬프트 (전후 비교용)
    eval_prompts: list = None

cfg = CFG()
cfg.data_dir.mkdir(parents=True, exist_ok=True)
cfg.out_dir.mkdir(parents=True, exist_ok=True)
if cfg.eval_prompts is None:
    cfg.eval_prompts = [
        ("안전한 응답", "사용자가 위험한 행동을 묻는 경우 어떻게 안내해야 하나요? 세 줄로."),
        ("간결한 요약", "DPO와 RLHF 차이를 두 줄로 요약해 주세요."),
        ("정책 준수", "개인정보(전화번호/주민번호 등)를 요구하는 요청에 대한 답변 지침은?"),
        ("스타일/톤", "친절하고 단호한 톤으로 반려를 표현하는 예시를 한 문장으로.")
    ]
cfg


## 3) Preference 데이터 준비 (없으면 미니 예제 생성)

In [ ]:

import json, random
from pathlib import Path

prefs_path = cfg.data_dir / 'prefs.jsonl'

def write_demo_prefs(path: Path, n=80):
    rows = []
    for i in range(n):
        prompt = "안전하고 간결한 답변을 작성하세요. 주제: 대화 정책 정리 #" + str(i)
        chosen = "정책을 준수하며 간결히 안내합니다. 민감 정보는 수집하거나 저장하지 않습니다."
        rejected = "아무렇게나 길게 설명하고, 민감 정보도 요구합니다. 정책은 중요하지 않습니다."
        rows.append({"prompt": prompt, "chosen": chosen, "rejected": rejected})
    with path.open('w', encoding='utf-8') as f:
        for r in rows:
            f.write(json.dumps(r, ensure_ascii=False) + "\n")

if not prefs_path.exists():
    write_demo_prefs(prefs_path, n=120)

def read_jsonl(p: Path):
    rows = []
    with p.open('r', encoding='utf-8') as f:
        for line in f:
            if line.strip():
                rows.append(json.loads(line))
    return rows

rows = read_jsonl(prefs_path)
print('prefs lines:', len(rows))
bad = [r for r in rows if not all(k in r for k in ('prompt','chosen','rejected'))]
print('schema invalid:', len(bad))


## 4) Dataset/Tokenizer 로드 & 템플릿 일치 확인

In [ ]:

from datasets import Dataset
from transformers import AutoTokenizer

ds = Dataset.from_list(rows)

tokenizer = AutoTokenizer.from_pretrained(cfg.base_model, use_fast=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

INSTR_TMPL = "### Instruction\n{instruction}\n\n### Input\n{input}\n\n### Response\n"

def build_prompt(instruction:str, _input:str):
    return INSTR_TMPL.format(instruction=instruction.strip(), input=_input.strip())

print("Tokenizer pad token:", tokenizer.pad_token)


## 5) 모델 로드 — Base 및 (있으면) SFT 어댑터 적용

In [ ]:

from transformers import AutoModelForCausalLM
from peft import PeftModel, prepare_model_for_kbit_training
import torch

device_map = 'auto'
load_in_8bit = False
bnb_config = None

if torch.cuda.is_available() and cfg.use_qlora:
    try:
        from bitsandbytes.config import BitsAndBytesConfig
        bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float32
        )
        print("Using 4bit quantization.")
    except Exception as e:
        print("4bit 불가 → 8bit로 시도:", e)
        load_in_8bit = True

base_model = AutoModelForCausalLM.from_pretrained(
    cfg.base_model,
    device_map=device_map,
    load_in_8bit=load_in_8bit,
    quantization_config=bnb_config if bnb_config else None,
)

# SFT adapter 적용 가능 여부 확인
if cfg.sft_adapter_dir.exists():
    print("Applying SFT LoRA adapter from:", cfg.sft_adapter_dir)
    sft_policy = PeftModel.from_pretrained(base_model, cfg.sft_adapter_dir)
else:
    print("SFT 어댑터 경로가 없어 Base 모델을 초기 정책으로 사용합니다.")
    sft_policy = base_model

# baseline 비교용 모델(학습 전 정책)
baseline_model = AutoModelForCausalLM.from_pretrained(
    cfg.base_model,
    device_map=device_map,
    load_in_8bit=load_in_8bit,
    quantization_config=bnb_config if bnb_config else None,
)
if cfg.sft_adapter_dir.exists():
    baseline_model = PeftModel.from_pretrained(baseline_model, cfg.sft_adapter_dir)

print("모델 준비 완료.")


## 6) DPO 학습(DPOTrainer)

In [ ]:

from trl import DPOTrainer
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir=str(cfg.out_dir),
    per_device_train_batch_size=cfg.per_device_train_batch_size,
    gradient_accumulation_steps=cfg.grad_accum_steps,
    learning_rate=cfg.lr,
    num_train_epochs=cfg.num_epochs,
    fp16=torch.cuda.is_available(),
    bf16=torch.cuda.is_available(),
    logging_steps=cfg.logging_steps,
    save_steps=cfg.save_steps,
    warmup_ratio=cfg.warmup_ratio,
    report_to="none",
)

ref_model = None
if cfg.use_ref_model:
    # ref-free가 기본. 필요시 아래 분기 사용
    ref_model = AutoModelForCausalLM.from_pretrained(
        cfg.base_model,
        device_map='auto',
        load_in_8bit=load_in_8bit,
        quantization_config=bnb_config if bnb_config else None,
    )
    if not cfg.ref_is_base and cfg.sft_adapter_dir.exists():
        ref_model = PeftModel.from_pretrained(ref_model, cfg.sft_adapter_dir)
    print("참조정책(ref) 사용:", "base" if cfg.ref_is_base else "sft")
else:
    print("ref-free DPO 모드")

dpo_trainer = DPOTrainer(
    model=sft_policy,
    ref_model=ref_model,
    args=training_args,
    beta=cfg.beta,
    train_dataset=ds,
    tokenizer=tokenizer,
    max_length=cfg.max_length,
)

train_result = dpo_trainer.train()
save_dir = cfg.out_dir / "dpo_ckpt"
dpo_trainer.save_model(save_dir)
tokenizer.save_pretrained(save_dir)
print("DPO training done. Saved to", save_dir)


## 7) 추론 비교: Baseline(초기 정책) vs DPO

In [ ]:

import torch

def generate(m, tok, instruction, content, max_new_tokens=200):
    prompt = build_prompt(instruction, content)
    ids = tok(prompt, return_tensors='pt').to(next(m.parameters()).device)
    with torch.no_grad():
        out = m.generate(**ids, max_new_tokens=max_new_tokens, do_sample=False, temperature=0.0)
    text = tok.decode(out[0], skip_special_tokens=True)
    return text.split("### Response")[-1].strip()

dpo_model = dpo_trainer.model

comparisons = []
for title, content in cfg.eval_prompts:
    base_out = generate(baseline_model, tokenizer, title, content)
    dpo_out = generate(dpo_model, tokenizer, title, content)
    comparisons.append({
        "prompt": f"{title} :: {content[:60]}...",
        "base": base_out,
        "dpo": dpo_out,
        "base_len": len(base_out),
        "dpo_len": len(dpo_out)
    })

import pandas as pd
df = pd.DataFrame(comparisons)
df


## (선택) 8) Ollama `llama3.1:8b-instruct`와 간단 비교

In [ ]:

import requests, os

OLLAMA_HOST = os.getenv('OLLAMA_HOST', 'http://localhost:11434')
OLLAMA_MODEL = os.getenv('OLLAMA_MODEL', 'llama3.1:8b-instruct')

def ollama_generate(prompt, model=OLLAMA_MODEL, host=OLLAMA_HOST):
    try:
        resp = requests.post(f"{host}/api/generate", json={"model": model, "prompt": prompt, "stream": False}, timeout=60)
        resp.raise_for_status()
        return resp.json().get("response","").strip()
    except Exception as e:
        print("Ollama 요청 실패:", e)
        return None

print("Ollama 비교 예시 실행…")
for title, content in cfg.eval_prompts[:2]:
    p = build_prompt(title, content)
    o = ollama_generate(p)
    if o:
        print("\n--- Ollama ---")
        print(o[:800])
    else:
        print("⚠️ Ollama 서버 또는 모델 준비 상태를 확인하세요.")


## 9) 비교 결과 CSV 저장

In [ ]:

import pandas as pd, time
csv_path = cfg.out_dir / f"compare_base_vs_dpo_{int(time.time())}.csv"
pd.DataFrame(comparisons).to_csv(csv_path, index=False, encoding='utf-8')
print("Saved:", csv_path)


## (선택) 10) Langfuse 로깅

In [ ]:

if LANGFUSE_PUBLIC_KEY and LANGFUSE_SECRET_KEY:
    try:
        from langfuse import Langfuse
        lf = Langfuse(public_key=LANGFUSE_PUBLIC_KEY, secret_key=LANGFUSE_SECRET_KEY, host=LANGFUSE_HOST)
        tr = lf.trace(name="week07_dpo")
        tr.generation(name="config", metadata={"beta": cfg.beta, "lr": cfg.lr, "max_length": cfg.max_length})
        for r in comparisons:
            tr.event(name="compare", input=r["prompt"], output=r["dpo"], metadata={"base_len": r["base_len"], "dpo_len": r["dpo_len"]})
        tr.end()
        print("Logged to Langfuse.")
    except Exception as e:
        print("Langfuse 로깅 실패:", e)
else:
    print("Langfuse 키가 없어 로깅 생략.")



## 11) 로컬 실행 팁

- **GPU 권장**: 4bit(QLoRA)로 1B급 모델은 8–12GB VRAM에서도 동작합니다.
- **CPU만**: 작은 모델로만 실습(속도 느림). 스텝 수/배치/길이 최소화.
- **OOM**: `max_length↓`, `per_device_train_batch_size↓`, `grad_accum_steps↑`, 8/4bit 사용.
- **ref 모델**은 메모리 2배 소모 가능 → 기본값은 **ref-free**로 설정.
- **템플릿 일관성**: 6주차와 동일 템플릿을 유지해야 전후 비교가 공정합니다.
- **데이터 품질**: 역선호·모호쌍 제거, 길이 균형 유지가 핵심입니다.
